## Задача

Реализовать алгоритм построения потока пенсий до достижения участниками договоров возраста 100 лет. В качестве входных данных используются следующие таблицы:
*	Таблица договоров участников;
*	Таблица сумм пенсий;
*	Таблица параметров расчета.

Формат выходной таблицы представлен на вкладке «Результат».


In [134]:
import pandas as pd
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [135]:
# participants_data = pd.read_csv("files/Договоры участников.csv", delimiter=';')
# pension_amounts = pd.read_csv("files/Суммы пенсий.csv", delimiter=';')
# calculation_params = pd.read_csv("files/Параметры расчёта.csv", delimiter=';')

excel_file = "files/Данные.xlsx"
participants_data = pd.read_excel(excel_file, sheet_name='Договоры участников')
pension_amounts = pd.read_excel(excel_file, sheet_name='Суммы пенсий')
calculation_params = pd.read_excel(excel_file, sheet_name='Параметры расчета', header=None)
calculation_params = calculation_params.set_index(0).transpose()

df_participants = pd.DataFrame(participants_data)
df_pensions = pd.DataFrame(pension_amounts)
df_params = pd.DataFrame(calculation_params)

In [136]:
# Вытаскиваем данные в отдельные переменные
report_date = pd.to_datetime(df_params['Отчетная дата'][1], format='%d.%m.%Y')
indexation_rate = df_params['Ставка индексации пенсии'][1]
max_age = int(df_params['Максимальный возраст, лет'][1])

# Стринг дат в datetime
df_participants['Дата рождения участника'] = pd.to_datetime(df_participants['Дата рождения участника'], format='%d.%m.%Y')

# Расчёт возраста участников на дату составления отчета
df_participants['Возраст на отчетную дату'] = (report_date - df_participants['Дата рождения участника']).dt.days // 365

# Слияние
df = pd.merge(df_participants, df_pensions, on='Номер договора')

def get_last_day_of_month(date):
    next_month = date + relativedelta(months=1)
    last_day = next_month - timedelta(days=next_month.day)
    return last_day

results = []

# График пенсионных выплат
for _, row in df.iterrows():
    contract_number = row['Номер договора']
    dob = row['Дата рождения участника']
    pension_age = row['Пенсионный возраст']
    initial_pension_amount = row['Установленный размер пенсии']

    # Начальная дата пенсии
    pension_start_date = dob + relativedelta(years=pension_age)
    pension_start_date = get_last_day_of_month(pension_start_date)

    payment_date = pension_start_date
    current_pension_amount = initial_pension_amount
    while (payment_date - dob).days // 365 < max_age:
        # Индексация в январе
        if payment_date.month == 1 and payment_date != pension_start_date:
            current_pension_amount *= (1 + indexation_rate)

        results.append({
            'Номер договора': contract_number,
            'Дата платежа': payment_date.strftime('%d.%m.%Y'),
            'Размер пенсии': round(current_pension_amount, 2)
        })

        next_payment_date = get_last_day_of_month(payment_date + relativedelta(months=1))
        # Проверка на превышение максимального возраста
        if (next_payment_date - dob).days // 365 >= max_age:
            results.append({
                'Номер договора': contract_number,
                'Дата платежа': next_payment_date.strftime('%d.%m.%Y'),
                'Размер пенсии': round(current_pension_amount, 2)
            })
            break
        payment_date = next_payment_date

df_results = pd.DataFrame(results)
df_results

,Номер договора,Дата платежа,Размер пенсии
0,10001,31.07.2021,30000.00
1,10001,31.08.2021,30000.00
2,10001,30.09.2021,30000.00
3,10001,31.10.2021,30000.00
4,10001,30.11.2021,30000.00
...,...,...,...
919,10002,30.09.2086,1277383.49
920,10002,31.10.2086,1277383.49
921,10002,30.11.2086,1277383.49
922,10002,31.12.2086,1277383.49


**Разделение для наглядности**

In [137]:
df_results[df_results['Номер договора'] == 10001]

,Номер договора,Дата платежа,Размер пенсии
0,10001,31.07.2021,30000.00
1,10001,31.08.2021,30000.00
2,10001,30.09.2021,30000.00
3,10001,31.10.2021,30000.00
4,10001,30.11.2021,30000.00
...,...,...,...
499,10001,28.02.2063,1642910.98
500,10001,31.03.2063,1642910.98
501,10001,30.04.2063,1642910.98
502,10001,31.05.2063,1642910.98


In [138]:
df_results[df_results['Номер договора'] == 10002]

,Номер договора,Дата платежа,Размер пенсии
504,10002,29.02.2052,50000.00
505,10002,31.03.2052,50000.00
506,10002,30.04.2052,50000.00
507,10002,31.05.2052,50000.00
508,10002,30.06.2052,50000.00
...,...,...,...
919,10002,30.09.2086,1277383.49
920,10002,31.10.2086,1277383.49
921,10002,30.11.2086,1277383.49
922,10002,31.12.2086,1277383.49


**Запись в исходный excel файл**

In [139]:
with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_results.to_excel(writer, sheet_name='Результат', index=False)